<a href="https://colab.research.google.com/github/smduarte/RC2021-labs/blob/main/tp2/RC2021_tp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reliable Data Transmission over an Unreliable Network<br> with the FT21 Protocol

# Goal

This assignment concerns the problem of delivering information reliably across a network where a degree of packet loss is expected. 

The goal is to understand how the performance of sliding window protocols is impacted by parameter choices, such as fixed timeouts, adaptative timeouts, windows sizes, as well as the flow control technique employed etc.

# Context

This assignment will use a simple file transfer protocol, between a *sender* and a *receiver?, implemented in the CNSS simulator. 

# FT21 Protocol

The FT21 protocol, developed for this assignment, allows for the reliable transfer of a file, over an unreliable network. This protocol comprises a total of 5 types of packets, namely `UPLOAD`, `DATA`, `FIN`, `ACK` and `ERROR`. 

## FT21 Packets

Each packet type is identified by an 1-byte numeric code.

The format and purpose of each packet is as follows:

* ***UPLOAD***

  `|0|optional-data|filename|`

  Initiates the transfer of the file with the given `filename. This packet has 0 as its implicit *sequence number*. 
  
* ***DATA***

  `|1|seqN|optional-data|data|`

  Represents a block of file data. The block is identified by a *sequence number* (`seqN´), starting at 1, for the first block of a file.

  `data` - the file block payload encoded as raw bytes. 

* ***FIN***

  `|2|optional-data|seqN|`

  Signals the file transfer is complete. The sequence number (`seqN`) should be 1 past the last block of the file.

* ***ACK***

  `|3|cSeqN|copied-optional-data|`

  Confirms correctly received packets.

  `cSeqN` - Represents a **cumulative sequence number** that acknowledges **all** packets up to and including the given value. 

* ***ERROR***

  `|4|errormessage|`

  Reports a fatal error.

The FT21 protocol allows the sender to include up to 255 bytes of arbitrary `optional-data` in its outgoing packets. The receiver will not interpret this data in any way. The only guarantee is that the `optional-data` portion of a packet is copied over to the `copied-optional-data` of the `ACK` packet it generates. 

The first byte of the `optional-data` of a packet contains the `length` of the (remaining) `optional-data` portion of the packet. A value of 0 means there is no 
`optional-data`, besides the initial byte. If `optional-data` bytes are present in the packet, their
meaning and format is up to the sender.

## FT21 File Transfer 

The figure below illustrates the packet exchange between the client and server, using the `Stop&Wait` protocol.


<img src="https://docs.google.com/drawings/d/e/2PACX-1vTfqFPbKxMIgy2Y4vWfXV7gxC96BtpLpgTBGjbNFgIK8N_ns-qgf6mQXfn27NPS1nDE5Zyg1QEeBKUF/pub?w=961&h=738" width="75%"></img>

To achieve a reliable file transfer, `UPLOAD`, `DATA` and `FIN` packets are retransmitted until acknowledged by the receiver.


## CNSS Simulation 

### FT21 Packets

[FT21Packet.java](https://github.com/smduarte/RC2021-labs/blob/main/tp2/RC2021-tp2/src/ft21/FT21Packet.java) is a Java class that encodes FT21Packets into arrays of bytes, which can be used as the payloads of CNSS [DataPackets](https://github.com/jlegatheaux/cnss/blob/master/src/cnss/simulator/DataPacket.java). 

Companion classes, in package ft21: [FT21_UploadPacket](https://github.com/smduarte/RC2021-labs/blob/master/tp2/RC2021-tp2/src/ft21/FT21_UploadPacket.java), [FT21_DataPacket](https://github.com/smduarte/RC2021-labs/blob/master/tp2/RC2021-tp2/src/ft21/FT21_DataPacket.java), [FT21_AckPacket](https://github.com/smduarte/RC2021-labs/blob/master/tp2/RC2021-tp2/src/ft21/FT21_AckPacket.java), [FT21_FinPacket](https://github.com/smduarte/RC2021-labs/blob/master/tp2/RC2021-tp2/src/ft21/FT21_FinPacket.java), [FT21_ErrorPacket](https://github.com/smduarte/RC2021-labs/blob/master/tp2/RC2021-tp2/src/ft21/FT21_ErrorPacket.java) model the default version of each specific FT21 packet type, without any optional data. These classes expose FT21 Packet fields as public **immutable** data. In these classes, the public constructor is the only way to assemble new FT21 packets.

The provided companion classes **can be changed** to include other fields as part of the optional-data portion of the packet, as exemplified below. Take notice that it
is assumed that the `optional-data` format is the same across all packet types.


In [ ]:
%%bash
mkdir -p src/ft21/recv
mkdir -p configs

In [ ]:
%%writefile src/ft21/FT21_UploadPacket2.java

package ft21;

public class FT21_UploadPacket2 extends FT21Packet {
    public final String filename;
    public final int optional_data_field1;
    public final int optional_data_field2;

    public FT21_UploadPacket(String filename, int optional_data_field1, int optional_data_field2) {
        super(PacketType.UPLOAD);
        super.putByte(2 * Integer.BYTES );
        super.putInt( optional_data_field1 );
        super.putInt( optional_data_field2 );
        super.putString(filename);
        this.filename = filename;
        this.optional_data_field1 = optional_data_field1;
        this.optional_data_field2 = optional_data_field2;
    }
    
    public String toString() {
        return String.format("UPLOAD<%s, %d, %d>", filename, optional_data_field1, optional_data_field2);
    }
}

Accordinly, the `FT21_AckPacket` class would have to be modified to decode and expose `optional_data_field1` and `optional_data_field1`. 


### FT21 Receiver

The supplied code already provides an implementation of the [receiver node](https://github.com/smduarte/RC2021-labs/blob/master/tp2/RC2021-tp2/src/ft21/recv/FT21Receiver.java). 

The receiver accepts a single integer parameter corresponding to its `receiver window size`. 

The receiver confirms packets using cummulative sequence numbers (`cSeqN`). If the `receiver window size` is larger
than 1 for DATA packets that fall outside the receiver window, the receiver generates ACK packets
with the cummulative sequence number negated (negative).

The receiver is implemented in package [ft21.recv](https://github.com/smduarte/RC2021-labs/blob/master/tp2/RC2021-tp2/src/ft21/recv/). **Nothing in this package may be modified for this assignment**. 

### Stop&Wait Example

  * ***RECEIVER***

In [ ]:
%%writefile src/ft21/recv/FT21Receiver.java

public class FT21Receiver extends FT21AbstractReceiverApplication {

    private int windowSize; // by default in blocks
    
    private SortedMap<Integer, byte[]> window = new TreeMap<>();

    private int nextSeqN;
    private String filename;
    private FileOutputStream fos;

    public FT21Receiver() {
        super(true, "   FT21Receiver");
    }

    @Override
    public int initialise(int now, int nodeId, Node self, String[] args) {
        super.initialise(now, nodeId, self, args);
        if( args.length != 1 ) {
            System.err.println( this.getClass().getSimpleName() + " missing windowSize argument [in config file]");
            System.exit(-1);
        }
        this.windowSize = Integer.valueOf(args[0]);
        this.fos = null;
        this.nextSeqN = 0;
        return 0;
    }

    @Override
    public void on_receive_upload(int now, int client, FT21_UploadPacket upload) {
        super.logPacket(now, upload);
        
        if (nextSeqN <= 1) {
            super.sendPacket(now, client, new FT21_AckPacket(0, upload.optional_data));
            nextSeqN = 1;
            window.clear();
            filename = upload.filename;
        } else
            super.sendPacket(now, client,
                new FT21_ErrorPacket("Unexpected packet type...[Already initiated a transfer...]"));
    }

    @Override
    public void on_receive_data(int now, int client, FT21_DataPacket block) {
        super.logPacket(now, block);

        // outside the window.
        if (block.seqN < nextSeqN || block.seqN >= nextSeqN + windowSize) {
            int cSeqN = (windowSize == 1 ? nextSeqN - 1 : -(nextSeqN - 1));
            super.sendPacket(now, client, new FT21_AckPacket( cSeqN, block.optional_data));			
        }
        else {
            window.putIfAbsent(block.seqN, block.data);

            //try to slide window and flush to disk.
            byte[] bytes;
            while ((bytes = window.remove(nextSeqN)) != null) {
                writeBlockToFile(bytes);
                nextSeqN += 1;
            }
            super.sendPacket(now, client, new FT21_AckPacket(nextSeqN - 1, block.optional_data));
        }
    }

    @Override
    public void on_receive_fin(int now, int client, FT21_FinPacket fin) {
        super.logPacket(now, fin);

        if (window.isEmpty() && nextSeqN == fin.seqN)
            super.printReport( now );

        super.sendPacket(now, client, new FT21_AckPacket(fin.seqN, fin.optional_data));
    }

    private void writeBlockToFile(byte[] data) {
        try {
            if (fos == null) fos = new FileOutputStream("copy-of-" + filename);
            fos.write(data);
        } catch (Exception x) {
            System.err.println("FATAL ERROR: " + x.getMessage());
            System.exit(-1);
        }
    }
}

  * ***SENDER***

In [ ]:
%%writefile src/FT21SenderSW.java

import java.io.*;
import cnss.simulator.*;
import ft21.*;

public class FT21SenderSW extends FT21AbstractSenderApplication {

    private static final int TIMEOUT = 1000;

    static int RECEIVER = 1;

    enum State {
        BEGINNING, UPLOADING, FINISHING, FINISHED
    };

    static int DEFAULT_TIMEOUT = 1000;

    private File file;
    private RandomAccessFile raf;
    private int BlockSize;
    private int nextPacketSeqN, lastPacketSeqN;

    private State state;
    private int lastPacketSent;

    public FT21SenderSW() {
        super(true, "FT21SenderSW");
    }

    public int initialise(int now, int node_id, Node nodeObj, String[] args) {
        super.initialise(now, node_id, nodeObj, args);

        raf = null;
        file = new File(args[0]);
        BlockSize = Integer.parseInt(args[1]);

        state = State.BEGINNING;
        lastPacketSeqN = (int) Math.ceil(file.length() / (double) BlockSize);

        lastPacketSent = -1;
        return 1;
    }

    public void on_clock_tick(int now) {
        boolean canSend = lastPacketSent < 0 || (now - lastPacketSent) > TIMEOUT;

        if (state != State.FINISHED && canSend)
            sendNextPacket(now);
    }

    private void sendNextPacket(int now) {
        switch (state) {
        case BEGINNING:
            super.sendPacket(now, RECEIVER, new FT21_UploadPacket(file.getName()));
            break;
        case UPLOADING:
            super.sendPacket(now, RECEIVER, readDataPacket(file, nextPacketSeqN));
            break;
        case FINISHING:
            super.sendPacket(now, RECEIVER, new FT21_FinPacket(nextPacketSeqN));
            break;
        case FINISHED:
        }

        lastPacketSent = now;
    }

    @Override
    public void on_receive_ack(int now, int client, FT21_AckPacket ack) {
        switch (state) {
        case BEGINNING:
            state = State.UPLOADING;
        case UPLOADING:
            nextPacketSeqN = ack.cSeqN + 1;
            if (nextPacketSeqN > lastPacketSeqN)
                state = State.FINISHING;
            break;
        case FINISHING:
            super.log(now, "All Done. Transfer complete...");
            super.printReport(now);
            state = State.FINISHED;
            return;
        case FINISHED:
        }
    }

    private FT21_DataPacket readDataPacket(File file, int seqN) {
        try {
            if (raf == null) raf = new RandomAccessFile(file, "r");

            raf.seek(BlockSize * (seqN - 1));
            byte[] data = new byte[BlockSize];
            int nbytes = raf.read(data);
            return new FT21_DataPacket(seqN, data, nbytes);
        } catch (Exception x) {
            throw new Error("Fatal Error: " + x.getMessage());
        }
    }
}


  * ***Configuration***

In [ ]:
%%writefile configs/config-2.1.txt
# A network with a sender node and a receiver node interconnected
# by a direct link. The link has 2 Mbps bandwidth and 20 ms latency


# uncomment if you want to see control algorithms traces
# parameter trace 


Node 0 1 cnss.lib.EndSystemControl FT21SenderSW earth.jpg 1000
Node 1 1 cnss.lib.EndSystemControl FT21Receiver 10

Link 0.0 1.0 2000000 20 0.25 0.0

  * ***Execution***

In [ ]:
%%bash
git clone https://github.com/jlegatheaux/cnss.git 2> /dev/null || git -C cnss pull
javac -d cnss-classes cnss/src/*/*/*.java

git clone https://github.com/smduarte/RC2021-labs.git /tmp/RC 2> /dev/null || git -C /tmp/RC pull
mv -f /tmp/RC/tp2/*/* . 2> /dev/null

javac -cp .:cnss-classes -d ft21-classes src/*/*/*.java src/*/*.java src/*.java

java -cp .:cnss-classes:ft21-classes cnss.simulator.Simulator configs/config-2.1.txt > results.txt && cat results.txt

---

# Deliverables

There are three deliverables:
* Delivery **A** (Mandatory): **GoBackN**
* Delivery **B** (Optional): **Selective Repeat**
* Delivery **C** (Optional): **Adaptative Timeouts**

Valid combinations are: **A**, **A+B**, **A+C**, **A+B+C**.

### A - GoBackN (Mandatory), up to 13 marks.

Implement the sliding window version of the FT21Protocol, using the **GoBackN** technique.

Your *sender* should be implemented in a class named `FT21SenderGBN`. It has to accept 3 arguments, in this order:

`filename` - the file being transferred;

`blocksize` - the size of file blocks sent in each `DATA` packet;

`windowsize` - the capacity of the window in number of blocks.

For testing purposes use the provided server FT21Receiver class. Your client should work correctly with the provided code. You must ***not change*** the receiver in any way. 

Use a fixed, default timeout value of 1000 ms.

Watch out for corner cases, such as: files that fit in a single block, files with length that is whole multiple of the blocksize.

Test your implementation against configurations [config-2.2](https://github.com/smduarte/RC2021-labs/blob/main/tp2/RC2021-tp2/configs/config-2.2.txt) and [config-2.3](https://github.com/smduarte/RC2021-labs/blob/main/tp2/RC2021-tp2/configs/config-2.3.txt)

Your solution should be as general as possible. It will be tested against a variety of configurations in addition to those provided.

Confirm the transfered file is identical to the original. You can use the `diff` command for that.

***Important Notes*** 
* Look at the implementation constraints at the end of this notebook. Violating them will penalize your final evaluation.

### B Selective Repeat (Optional), up to 4 marks

Improve your previous delivery by using the techniques introduced by the **Selective Repeat** version of the protocol. 

Your *sender* should be implemented in a class named `FT21SenderSR`.

Make use of `optional-data` in packets.

Pay attention to how the receiver signals that a DATA packet fell outside the receiver window. 

Note: The receiver is already compatible with a Selective Repeat sender but requires a window size larger than 1 to be effective. 

Use a fixed default timeout value of 1000 ms. 

Using the provided configuration files, compare the results of this version against those for the GBN version of the first deliverable.

Test your implementation against configuration [config-2.4](https://github.com/smduarte/RC2021-labs/blob/main/tp2/RC2021-tp2/configs/config-2.4.txt)

***Important Notes*** 
* Look at the implementation constraints at the end of this notebook. Violating them will penalize your final evaluation.

### C - Adaptative timeouts (Optional), up to 3 marks

Your *sender* should be implemented in a class named `FT21SenderGBN_DT` or `FT21SenderSR_DT`.

Replace the fixed timeout value with a dynamic value derived from an estimate of the network RTT between the sender and receiver. 

Compare the results of this version against those for the fixed timeout versions of the previous deliverables, using the provided configuration files.

## Statistics

A statistics report should be presented at the end of each file transfer. 

Refer to the provided reference Stop&Wait version [FT21SenderSW](https://github.com/smduarte/RC2021-labs/blob/main/tp2/RC2021-tp2/src/FT21SenderSW.java) to see how to produce the statistics report.

## Materials 

[CNSS simulator](https://github.com/jlegatheaux/cnss)

[Stop&Wait using the FT21 Protocol](https://github.com/smduarte/RC2021-labs/tree/main/tp2)

### Implementation Constraints

You are allowed to send at most 1 packet per invocation of the `on_timeout`, `on_clock_tick`, `on_receive_*`.
upcalls.